In [1]:
%pip install -q -U anthropic
%pip install -q -U lightning
%pip install -q -U bitsandbytes
%pip install -q -U transformers
%pip install -q -U peft
%pip install -q -U accelerate
%pip install -q -U wandb
%pip install -q -U datasets
%pip install -q -U prodigyopt
%pip install pdf2image
%pip install -q -U protobuf==3.20.0
!apt-get update --yes
!apt-get install -y poppler-utils

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the pac

In [1]:
import torch
import os
import json
import random
from transformers import AutoProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
from transformers import DonutProcessor
from torch.utils.data import DataLoader
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
import os, torch, torch.nn as nn, torch.utils.data as data, torchvision as tv
import lightning as L
from transformers import DonutProcessor, VisionEncoderDecoderModel
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
import wandb
import os
from IPython.display import Image
from tqdm.auto import tqdm
from IPython.display import Image
import pickle
import torchvision.transforms as T
from datasets import Dataset
from datasets import IterableDataset
from tqdm.auto import tqdm
import unicodedata
import os
import gc
from torch.utils.data import Dataset
import torchvision as tv
import torchvision.transforms as T
from transformers import DonutProcessor
import random
from torch.utils.data import Subset
from peft import LoraConfig
from torch.optim.lr_scheduler import LinearLR
import lightning as L
from prodigyopt import Prodigy
from datetime import datetime
import torch.multiprocessing as mp
from torch.cuda.amp import autocast, GradScaler
from pdf2image import convert_from_path, convert_from_bytes
from transformers import VisionEncoderDecoderModel, DonutProcessor
from pytorch_lightning.loggers import WandbLogger
from pdf2image import convert_from_path
from torchvision.transforms import ToTensor

2024-10-28 16:58:41.851441: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 16:58:41.851490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 16:58:41.852631: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 16:58:41.859449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 16:58:42.775558: W tensorflow/compiler/tf2

In [2]:
%pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
DATASET_PATH = "./documents"
PDF_PATH = os.path.join(DATASET_PATH, "german_pdf_files")
IMAGE_PATH = os.path.join(DATASET_PATH, "german_img_files")


class DocMeta:
    def __init__(self, dataset_path=DATASET_PATH, image_path=IMAGE_PATH):
        self.dataset_path = dataset_path
        self.image_path = image_path
        self.extraction = self.load_extraction(self.dataset_path)
        self.mapping = self.load_mapping(self.image_path)

    def get_random_urls(self, num=100):
        return random.sample(list(self.extraction.keys()), num)

    def load_extraction(self, path):
        with open(os.path.join(path, "extraction.json"), "r") as f:
            return json.load(f)

    def load_mapping(self, path):
        with open(os.path.join(path, "mapping.json"), "r") as f:
            return json.load(f)

    def get_image_path(self, url):
        return os.path.join(self.image_path, self.mapping[url])

    def get_image(self, url):
        img_path = self.get_image_path(url)
        return Image(filename=img_path)


doc_meta = DocMeta()

In [3]:
class DocMetaDataset(Dataset):
    def __init__(self, doc_meta, output_dir, ocr=False):
        self.doc_meta = doc_meta
        self.task_start_token = "<s>"
        self.eos_token = "</s>"
        self.processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
        self.processor.tokenizer.add_special_tokens(
            {"additional_special_tokens": [self.task_start_token, self.eos_token]}
        )
        self.processor.feature_extractor.size = [2160, 3840]
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        self.data = self._prepare_data()
        self.ocr = ocr

    def _prepare_data(self):
        data = []
        for url in tqdm(
            list(self.doc_meta.extraction.keys()), desc="Preparing dataset"
        ):
            main_author = self.doc_meta.extraction[url]["metadata"]["main_author"]

            transcript = self.doc_meta.extraction[url]["transcript"]

            transcript = self.task_start_token + transcript + self.eos_token

            if not main_author is None:

                if "name" in main_author:

                    author = main_author["name"]

                elif "first_name" in main_author and "last_name" in main_author:

                    author = ""
                    if main_author["first_name"] is not None:
                        author += main_author["first_name"]
                    if main_author["last_name"] is not None:
                        author += " " + main_author["last_name"]

                elif "department" in main_author:

                    author = main_author["department"]

                else:
                    continue
            else:
                continue

            author = self.task_start_token + author + self.eos_token
            image_path = self.doc_meta.get_image_path(url)

            image_data = tv.io.read_image(unicodedata.normalize("NFC", image_path))
            res = tv.transforms.functional.get_image_size(image_data)

            if res[0] > 2160 or res[1] > 3840:
                continue

            if author.replace(" ", "") == "<s></s>":
                continue

            if len(author) > 80:
                continue

            data.append((image_path, author, url, transcript))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, author, url, transcript = self.data[idx]

        image_data = tv.io.read_image(unicodedata.normalize("NFC", image_path))
        image_data = image_data.permute(1, 2, 0)

        pixel_values = self.processor(image_data, return_tensors="pt").pixel_values
        pixel_values = pixel_values.squeeze()

        if not self.ocr:
            input_ids_author = (
                self.processor.tokenizer(
                    author,
                    add_special_tokens=False,
                    padding="max_length",
                    truncation=True,
                    max_length=80,
                    return_tensors="pt",
                )["input_ids"]
                .squeeze(0)
                .long()
            )

            author_target = input_ids_author.clone()
            author_target[author_target == self.processor.tokenizer.pad_token_id] = -100
            return {
                "pixel": pixel_values,
                "target": author_target,
                "url": url,
            }

        if self.ocr:
            input_ids_transcript = (
                self.processor.tokenizer(
                    transcript,
                    add_special_tokens=False,
                    padding="max_length",
                    truncation=True,
                    max_length=300,
                    return_tensors="pt",
                )["input_ids"]
                .squeeze(0)
                .long()
            )
            transcript_target = input_ids_transcript.clone()
            transcript_target[
                transcript_target == self.processor.tokenizer.pad_token_id
            ] = -100
            return {
                "pixel": pixel_values,
                "target": transcript_target,
                "url": url,
            }

    def get_split(self, train_ratio=0.8, val_ratio=0.1, seed=None):
        total_size = len(self.data)
        train_size = int(train_ratio * total_size)
        val_size = int(val_ratio * total_size)
        test_size = total_size - train_size - val_size  # Ensure all data is used

        indices = list(range(total_size))
        if seed is not None:
            random.Random(seed).shuffle(indices)
        else:
            random.shuffle(indices)

        train_indices = indices[:train_size]
        val_indices = indices[train_size : train_size + val_size]
        test_indices = indices[train_size + val_size :]

        train_dataset = Subset(self, train_indices)
        val_dataset = Subset(self, val_indices)
        test_dataset = Subset(self, test_indices)

        return train_dataset, val_dataset, test_dataset

In [4]:
dataset = DocMetaDataset(doc_meta, "./")

/usr/local/lib/python3.11/dist-packages/transformers/models/donut/processing_donut.py:208: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(


Preparing dataset:   0%|          | 0/4702 [00:00<?, ?it/s]

In [5]:
train, val, test = dataset.get_split(0.8, 0.18)
len(train), len(val), len(test)

(2892, 650, 74)

In [6]:
dataset.ocr = False

In [7]:
batch_size = 1
num_workers = 0

train_dataloader = DataLoader(
    train, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True
)

val_dataloader = DataLoader(
    val, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

test_dataloader = DataLoader(
    test, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

In [14]:
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

model.decoder.resize_token_embeddings(len(dataset.processor.tokenizer))
model.config.pad_token_id = dataset.processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = dataset.processor.tokenizer.convert_tokens_to_ids(
    ["<s>"]
)[0]


# peft_config = LoraConfig(
#    lora_alpha=128, lora_dropout=0.05, r=256, bias="none", target_modules="all-linear"
# )
#
# model = get_peft_model(model, peft_config)

model = model.to("cuda")

# model.print_trainable_parameters()

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    2560,
    1920
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "transformers_version": "4.46.0",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add

In [9]:
torch.set_float32_matmul_precision("medium")

In [11]:
wandb.login()

model.safetensors:   0%|          | 0.00/809M [00:00<?, ?B/s]

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
# compiled_model = torch.compile(model) # not working

In [15]:
example = next(iter(test_dataloader))
class ModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, pixel_values, labels):
        outputs = self.model(
            pixel_values=pixel_values,
            labels=labels
        )
        return outputs.loss, outputs.logits

# 5. Try tracing with the wrapper
wrapped_model = ModelWrapper(model)
model_traced = wrapped_model.to("cuda")
wrapped_model = wrapped_model.eval() # https://github.com/pytorch/pytorch/issues/23999#issuecomment-581687224
# NOTICE: Pruning does not work with tracing => trace after pruning!!
#with autocast():
#    with torch.no_grad():
#        model_traced = torch.jit.trace(
#            wrapped_model.to("cuda"),
#            (
#                example["pixel"].to("cuda"),
#                example["target"].to("cuda")
#            ),
#            check_trace=False # TODO: Hides errors: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error: Scalars are not close!
#        )

In [16]:
model = model_traced

# Quantization


In [ ]:
#from torch._export import capture_pre_autograd_graph
#exported_model = capture_pre_autograd_graph(model, (example["pixel"].to("cuda"), example["target"].to("cuda")))

In [ ]:
#exported_model = torch.export.export_for_training(model, (example["pixel"].to("cuda"), example["target"].to("cuda"))).module()

# Training

In [17]:
scaler = GradScaler()
optimizer = Prodigy(model.parameters(), lr=1.0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=10 * len(train_dataloader)
)


run = wandb.init(
    project="donut_torch",
)

accumulate_steps = 10

def train_one_epoch(epoch_index):
    running_loss = 0.0
    last_loss = 0.0

    for i, data in enumerate(train_dataloader):
        batch = data

        with torch.enable_grad():
            with autocast():
                loss, _ = model(
                    pixel_values=batch["pixel"].to("cuda"),
                    labels=batch["target"].to("cuda"),
                )

        # https://pytorch.org/docs/stable/amp.html#gradient-scaling
        scaler.scale(loss).backward()

        # accumulate gradients
        if (i + 1) % accumulate_steps == 0:
            scaler.step(optimizer)
            scheduler.step()
            scaler.update()  # why?
            optimizer.zero_grad()


        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 10  # loss per batch
            print("  batch {} loss: {}".format(i + 1, last_loss))
            wandb.log({"train_loss": last_loss})
            running_loss = 0.0

        # Test loss
        if i % 200 == 190:
            running_vloss = 0
            with torch.no_grad():
                for i, vdata in enumerate(test_dataloader):
                    vbatch = vdata
                    with autocast():
                        loss,_ = model(
                            pixel_values=vbatch["pixel"].to("cuda"),
                            labels=vbatch["target"].to("cuda"),
                        )
                        vloss = loss
                    running_vloss += vloss
            avg_vloss = running_vloss / (i + 1)
            print("  vloss: {}".format(avg_vloss))
            wandb.log({"v_loss": avg_vloss})


    return last_loss

train_loss,▁█▃▂
train_loss,7.67332


In [18]:
# TODO: epochs too long

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.0


for epoch in range(EPOCHS):
    print("EPOCH {}:".format(epoch_number + 1))

    model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    running_vloss = 0.0

    model.eval()

    with torch.no_grad():
        for i, vdata in enumerate(val_dataloader):
            vbatch = vdata
            with autocast():
                loss, _ = model(
                    pixel_values=vbatch["pixel"].to("cuda"),
                    labels=vbatch["target"].to("cuda"),
                )
                vloss = loss
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print("LOSS train {} valid {}".format(avg_loss, avg_vloss))
    wandb.log({"val_loss": avg_vloss})

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        print("Saving model ...")
        best_vloss = avg_vloss
        model_path = "model_{}_{}".format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)
        wandb.save(model_path)

    epoch_number += 1

EPOCH 1:


# Pruning

In [58]:
from torch.nn.utils import prune


In [59]:
parameters_to_prune = [
    (module, 'weight')
    for name, module in model.named_modules()
    if hasattr(module, 'weight')
]



In [60]:
parameters_to_prune

[(Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4)), 'weight'),
 (LayerNorm((128,), eps=1e-05, elementwise_affine=True), 'weight'),
 (LayerNorm((128,), eps=1e-05, elementwise_affine=True), 'weight'),
 (Linear(in_features=128, out_features=128, bias=True), 'weight'),
 (Linear(in_features=128, out_features=128, bias=True), 'weight'),
 (Linear(in_features=128, out_features=128, bias=True), 'weight'),
 (Linear(in_features=128, out_features=128, bias=True), 'weight'),
 (LayerNorm((128,), eps=1e-05, elementwise_affine=True), 'weight'),
 (Linear(in_features=128, out_features=512, bias=True), 'weight'),
 (Linear(in_features=512, out_features=128, bias=True), 'weight'),
 (LayerNorm((128,), eps=1e-05, elementwise_affine=True), 'weight'),
 (Linear(in_features=128, out_features=128, bias=True), 'weight'),
 (Linear(in_features=128, out_features=128, bias=True), 'weight'),
 (Linear(in_features=128, out_features=128, bias=True), 'weight'),
 (Linear(in_features=128, out_features=128, bias=True), 'weig

In [61]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.3, # remove 30% of all weights
)

In [32]:
def calculate_global_sparsity(model):
    total_zeros = 0
    total_elements = 0
    
    # Sum up zeros and elements across all Linear layers in BERT
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear) and "layer" in name:
            total_zeros += torch.sum(module.weight == 0)
            total_elements += module.weight.nelement()
    
    sparsity = 100.0 * float(total_zeros) / float(total_elements)
    print(f"Global sparsity: {sparsity:.2f}%")
    return sparsity

calculate_global_sparsity(model)

Global sparsity: 75.23%


75.23425502848005

# Upload

In [15]:
from huggingface_hub import login

login()

In [16]:
# does not work when using peft LoRA
model.push_to_hub("doc_meta")

# Inference


In [21]:
task_start_token = "<s>"
eos_token = "</s>"
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
processor.tokenizer.add_special_tokens(
    {"additional_special_tokens": [task_start_token, eos_token]}
)

processor.feature_extractor.size = [2160, 3840]
model = VisionEncoderDecoderModel.from_pretrained("sodowo/doc_meta")

model.decoder.resize_token_embeddings(len(processor.tokenizer))
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(
    ["<s>"]
)[0]

In [3]:
eval_model = model.eval()

In [4]:
pdf_path = "./uni_hd.pdf"
image_data = convert_from_path(pdf_path, fmt="jpeg", first_page=1, last_page=1)[0]
image_path = pdf_path.replace(".pdf", ".jpg")
image_data.save(image_path)

In [5]:
image_data = tv.io.read_image(unicodedata.normalize("NFC", image_path))
image_data = image_data.permute(1, 2, 0)

In [6]:
pixel_values = processor(image_data, return_tensors="pt").pixel_values
pixel_values = pixel_values.squeeze()

In [7]:
eval_model = eval_model.to("cuda")
with torch.no_grad():
    generated_ids = model.generate(pixel_values.unsqueeze(0).to("cuda"))

In [8]:
generated_ids

In [9]:
predicted_label = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [10]:
predicted_label

# ONNX

In [14]:
%pip install --quiet --upgrade onnxruntime onnx onnxscript

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.3 which is incompatible.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 5.28.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [16]:
torch.save(model, "_model.pt")

In [17]:
model = torch.load("_model.pt")

In [18]:
model = model.eval()

In [20]:
example = next(iter(test_dataloader))
data = {"pixel_values": example["pixel"].to("cuda"), "labels": example["target"].to("cuda")}
with autocast():
    res = model(**data)
res

Seq2SeqLMOutput(loss=tensor(6.6855, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[45.7188, -1.1904, 44.8438,  ..., 43.2188, 40.2812, 38.3438],
         [44.5000, -1.1660, 42.9375,  ..., 41.8125, 38.6250, 36.9375],
         [32.2500, -1.1318, 41.5625,  ..., 40.3438, 33.8438, 33.4062],
         ...,
         [13.8516, -0.5020, 20.7188,  ..., 21.1094, 19.6406, 18.2188],
         [13.9141, -0.5015, 20.7031,  ..., 21.1094, 19.6562, 18.2500],
         [13.7891, -0.5029, 20.5625,  ..., 20.9062, 19.4531, 18.0469]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[ 5.1318e-01,  1.2129e+00, -2.4727e+00,  ...,  8.1689e-01,
            7.1631e-01,  2.4731e-01],
          [ 8.2275e-01,  1.1855e+00, -2.1152e+00,  ...,  6.0010e-01,
            8.2373e-01,  3.5645e-01],
          [-1.1890e-01,  4.4043e-01,  2.8882e-01,  ..., -3.0762e-01,
            1.6736e-01,  1.6821e-01],
          ...,
          [-2.0176e+00, -6.2256e-01,  

In [21]:
example = next(iter(test_dataloader))
data = {"pixel_values": example["pixel"].to("cuda"), "labels": example["target"].to("cuda")}

PATH = "test_model.onnx"
with torch.no_grad():
    torch.onnx.export(model, data, PATH, verbose=True)

# TODO: Model converts tensor to python boolean => not tracable => some values (related to image resolution and size) are hardcoded


/usr/local/lib/python3.11/dist-packages/transformers/models/donut/modeling_donut_swin.py:265: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if width % self.patch_size[1] != 0:
/usr/local/lib/python3.11/dist-packages/transformers/models/donut/modeling_donut_swin.py:268: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if height % self.patch_size[0] != 0:
/usr/local/lib/python3.11/dist-packages/transformers/models/donut/modeling_donut_swin.py:575: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record t

Exported graph: graph(%input.1 : Float(1, 3, 3840, 2160, strides=[24883200, 8294400, 2160, 1], requires_grad=0, device=cuda:0),
      %labels : Long(1, 80, strides=[80, 1], requires_grad=0, device=cuda:0),
      %encoder.embeddings.patch_embeddings.projection.weight : Float(128, 3, 4, 4, strides=[48, 16, 4, 1], requires_grad=1, device=cuda:0),
      %encoder.embeddings.patch_embeddings.projection.bias : Float(128, strides=[1], requires_grad=1, device=cuda:0),
      %encoder.embeddings.norm.weight : Float(128, strides=[1], requires_grad=1, device=cuda:0),
      %encoder.embeddings.norm.bias : Float(128, strides=[1], requires_grad=1, device=cuda:0),
      %encoder.encoder.layers.0.blocks.0.layernorm_before.weight : Float(128, strides=[1], requires_grad=1, device=cuda:0),
      %encoder.encoder.layers.0.blocks.0.layernorm_before.bias : Float(128, strides=[1], requires_grad=1, device=cuda:0),
      %encoder.encoder.layers.0.blocks.0.attention.self.query.bias : Float(128, strides=[1], requi

KeyboardInterrupt: 

wandb: 🚀 View run frosty-haze-26 at: https://wandb.ai/jonasg/donut_torch/runs/7xk2cii6
wandb: Find logs at: wandb/run-20241028_083330-7xk2cii6/logs


In [ ]:
import onnx


model = onnx.load("test_model_quantized.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)
